In [1]:
import pandas as pd
import json
from neo4j import GraphDatabase
from datetime import datetime
import re
from networkx.readwrite import json_graph
import networkx as nx

### Connect neo4j

In [2]:
#connect neo4j
url = "bolt://localhost:7687"
user = 'neo4j'
password = 'bnm@1234' 
driver = GraphDatabase.driver(url, auth=(user, password))

#udf to run neo4j query
def runNeo4j(query, p = True):
    resultList = []
    with driver.session() as session:
        res = session.run(query)
        if p:
            for i in res:
                resultList.append(i)
                print(i[0])
        else:
            for i in res:
                resultList.append(i)
            return resultList

### UDF

In [3]:
def buildNetworkDict(x):
    start = datetime.now()
    #instantiate a dict
    graphDict = {}
    graphDict['nodes'] = []
    graphDict['links'] = []
    #check if nodes neo id is unique
    nodeIdList = []
    #check if rel id is unique
    relIdList = []

    for i in x:
        for category in ['a', 'b', 'r']:
            #for nodes information
            #sender or receiver
            if category in ['a', 'b']:
                #use neo4j id
                nodesId = i[category].id
                if nodesId not in nodeIdList:
                    nodeIdList.append(nodesId)
                    label = next(iter((i)[category].labels))
                    tempDict = dict(i[category])
                    tempDict['label'] = label
                    tempDict['NeoId'] = nodesId
                    graphDict['nodes'].append(tempDict)

            #for relationship
            elif category == 'r':
    #             relType = i['r'][0].type      
    #             transId = i['r'][0].id
    #             linkTempDict['source'] = senderId
    #             linkTempDict['target'] = receiverId
    #             linkTempDict['type'] = relType
    #             linkTempDict['transId'] = transId

                #loop throu rel
                for j in i['r']:
                    linkTempDict = {}
                    relId = j.id
                    if relId not in relIdList:
                        relIdList.append(relId)
                        linkTempDict['source'] = j.nodes[0].id
                        linkTempDict['target'] =j.nodes[1].id
                        linkTempDict['type'] = j.type
                        linkTempDict['transId'] = relId            
                        graphDict['links'].append(linkTempDict)

    #remove duplicated nodes and links
    #noDupLinks = list({v['source']:v for v in graphDict['links']}.values())
    #noDupNodes =  list({v['NeoId']:v for v in graphDict['nodes']}.values())

    #build graphDict without duplicates
    tempGraphDict = {}
    tempGraphDict['nodes'] = graphDict['nodes']
    tempGraphDict['links'] = graphDict['links']
    # tempGraphDict['links'] = noDupLinks
    time = datetime.now() - start
    print("Estimated time: %s"%time)
    return tempGraphDict

### Read file

In [5]:
csvFile = "D:/Users/figohjs/Documents/Network/data/raw/subject.csv"
df = pd.read_csv(csvFile)

offenseList = df['Offence'].unique()

In [6]:
#top 10 offence
df['Offence'].value_counts()[:10]

Fraud                           1428
Corruption And Bribery           821
Tax Evasion                      594
Terrorism and TF                 418
Others                           326
Organised Crimes                 119
Proliferation Financing          112
Illicit Trafficking Of Drugs      98
Money Laundering                  76
Corruption                        70
Name: Offence, dtype: int64

### Scam

In [ ]:
#number of scam cases
scamOffense = [i for i in offenseList if re.search("Scam", str(i), re.I)]
dfScam = df[df['Offence'].isin(scamOffense)].copy()
numScamOffense = dfScam.shape[0]
print("Number of scam cases: %s"%numScamOffense)
print("Missing ID: %s"%dfScam[pd.isnull(dfScam['ID'])].shape[0])
print("Number of unique names: %s"%len(dfScam['Name'].unique()))

In [ ]:
#clean name
dfScam['CleanName'] = dfScam['Name'].replace(to_replace = ['\(.*\)', '\.'],
                                             value = '',  regex = True).str.strip()
scamNameList  = dfScam['CleanName'].unique()

In [ ]:
#case insensitive search
firstStr = ['a.pName =~ "(?i)'  + i + '"' for i in scamNameList]
firstStr = ' or '.join(firstStr)
# secondStr = ['b.pName =~ "(?i)'  + i + '"' for i in scamNameList]
# secondStr = ' or '.join(secondStr)

hops = 3
start = datetime.now()
query1 = "MATCH (a)-[r*..%s]-(b) "%hops
query2 = "WHERE (%s)"%(firstStr)
query3 = "RETURN a, r, b"
finalQuery = query1 + query2 + query3
result = runNeo4j(finalQuery, p = False)
time = datetime.now() - start
print("Estimated time: %s"%time)

In [ ]:
networkResult = buildNetworkDict(result)

print("Number of unique nodes: %s"%len(networkResult['nodes']))
print("Number of links: %s"%len(networkResult['links']))

In [ ]:
#build network graph from dict of dict
networkGraph = json_graph.node_link_graph(networkResult)
#find number of subgraphs
subGraphs = nx.connected_components(networkGraph)
#filter out subgraphs with only one node
subGraphsList = [i for i in list(subGraphs) if len(i)!=1]
print("Number of subgraphs: %s"%len(list(subGraphsList)))

In [ ]:
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2021-02-19_ScamNetwork52Names.json"
with open(jsonFile, 'w') as file:
    json.dump(networkResult, file) 

In [4]:
#for 5 hops
filename = "D:/Users/figohjs/Documents/Network/data/interim/2021-03-03_ScamId.csv"
dfScam2 = pd.read_csv(filename)

idList = list(dfScam2['0'].values)

In [ ]:
hops = 5
start = datetime.now()
query1 = "WITH " + str(idList) + ' as idList '
query2 = "MATCH (a:Entity)-[r*..%s]-(b:Entity) "%hops
query3 = "WHERE any(i in a.pIdList where i in idList) "
query4 = "RETURN a, r, b"
finalQuery = query1 + query2 + query3 + query4
result = runNeo4j(finalQuery, p = False)
time = datetime.now() - start
print("Estimated time: %s"%time)

In [10]:
len(result)

0

In [ ]:
#first 100 names
networkResult = buildNetworkDict(result)

print("Number of unique nodes: %s"%len(networkResult['nodes']))
print("Number of links: %s"%len(networkResult['links']))

### Corruption

In [ ]:
dfCorruption = df[df['Offence'] == 'Corruption And Bribery'].copy()
numCorruptionOffense = dfCorruption.shape[0]
print("Number of scam cases: %s"%numCorruptionOffense)
print("Missing ID: %s"%dfCorruption[pd.isnull(dfCorruption['ID'])].shape[0])
print("Number of unique names: %s"%len(dfCorruption['Name'].unique()))

In [ ]:
dfCorruption.head()

In [ ]:
dfCorruption['CleanName'] = dfCorruption['Name'].replace(to_replace = ['\(M\)', '\([\d\-]+\)', '\-', '\.'],
                                                            value = '',  regex = True)\
                                                .replace(to_replace = ["\&"],
                                                            value = 'AND', regex = True).str.strip()

dfCorruption['FinalName'] = dfCorruption.apply(lambda x:x['CleanName'] if pd.isnull(x['ID'])
                                                        else '', axis = 1)

dfCorruption['ID'] = dfCorruption['ID'].replace(to_replace = ["\-"],
                                               value = '', regex = True)

CorruptionNameList = [i for i in dfCorruption['FinalName'] if i!='']

CorruptionIDList = list(dfCorruption[pd.notnull(dfCorruption['ID'])]['ID'].values)

In [ ]:
#case insensitive search
firstStr = ['a.pName =~ "(?i)'  + i + '"' for i in CorruptionNameList[:200]]
firstStr = ' or '.join(firstStr)

hops = 3
start = datetime.now()
query1 = "WITH " + str(CorruptionIDList) + ' as idList '
query2 = "MATCH (a)-[r*..%s]-(b) "%hops
# query3 = "WHERE (%s) or any(i in a.pIdList where i in idList) "%(firstStr)
query3 = "WHERE (%s) "%(firstStr)
query4 = "RETURN a, r, b"
finalQuery = query1 + query2 + query3 + query4
result = runNeo4j(finalQuery, p = False)
time = datetime.now() - start
print("Estimated time: %s"%time)

In [ ]:
len(result)

In [ ]:
# import pickle
# l = [1,2,3,4]
# with open("test.txt", "wb") as fp:   #Pickling
#     pickle.dump(l, fp)
    
# with open("test.txt", "rb") as fp:   # Unpickling
#     b = pickle.load(fp)

In [ ]:
type(result[0])

In [ ]:
#case insensitive search
firstStr = ['a.pName =~ "(?i)'  + i + '"' for i in CorruptionNameList[100:200]]
firstStr = ' or '.join(firstStr)

hops = 3
start = datetime.now()
query1 = "WITH " + str(CorruptionIDList) + ' as idList '
query2 = "MATCH (a)-[r*..%s]-(b) "%hops
# query3 = "WHERE (%s) or any(i in a.pIdList where i in idList) "%(firstStr)
query3 = "WHERE (%s) "%(firstStr)
query4 = "RETURN a, r, b"
finalQuery = query1 + query2 + query3 + query4
result = runNeo4j(finalQuery, p = False)
time = datetime.now() - start
print("Estimated time: %s"%time)

In [ ]:
len(result)

In [ ]:
#first 100 names
networkResult = buildNetworkDict(result)

print("Number of unique nodes: %s"%len(networkResult['nodes']))
print("Number of links: %s"%len(networkResult['links']))

In [ ]:
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2021-02-28_CorruptionNetwork100Names.json"
with open(jsonFile, 'w') as file:
    json.dump(networkResult, file) 

In [ ]:
len(result)

In [ ]:
#second 100 names
networkResult = buildNetworkDict(result)

print("Number of unique nodes: %s"%len(networkResult['nodes']))
print("Number of links: %s"%len(networkResult['links']))

In [ ]:
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2021-02-28_CorruptionNetwork100Names2.json"
with open(jsonFile, 'w') as file:
    json.dump(networkResult, file) 

In [ ]:
#case insensitive search
firstStr = ['a.pName =~ "(?i)'  + i + '"' for i in CorruptionNameList[200:300]]
firstStr = ' or '.join(firstStr)

hops = 3
start = datetime.now()
query1 = "WITH " + str(CorruptionIDList) + ' as idList '
query2 = "MATCH (a)-[r*..%s]-(b) "%hops
# query3 = "WHERE (%s) or any(i in a.pIdList where i in idList) "%(firstStr)
query3 = "WHERE (%s) "%(firstStr)
query4 = "RETURN a, r, b"
finalQuery = query1 + query2 + query3 + query4
result = runNeo4j(finalQuery, p = False)
time = datetime.now() - start
print("Estimated time: %s"%time)

In [ ]:
#first 100 names
networkResult = buildNetworkDict(result)

print("Number of unique nodes: %s"%len(networkResult['nodes']))
print("Number of links: %s"%len(networkResult['links']))

In [ ]:
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2021-02-28_CorruptionNetwork100Names3.json"
with open(jsonFile, 'w') as file:
    json.dump(networkResult, file) 

In [ ]:
#case insensitive search
firstStr = ['a.pName =~ "(?i)'  + i + '"' for i in CorruptionNameList[300:400]]
firstStr = ' or '.join(firstStr)

hops = 3
start = datetime.now()
query1 = "WITH " + str(CorruptionIDList) + ' as idList '
query2 = "MATCH (a)-[r*..%s]-(b) "%hops
# query3 = "WHERE (%s) or any(i in a.pIdList where i in idList) "%(firstStr)
query3 = "WHERE (%s) "%(firstStr)
query4 = "RETURN a, r, b"
finalQuery = query1 + query2 + query3 + query4
result = runNeo4j(finalQuery, p = False)
time = datetime.now() - start
print("Estimated time: %s"%time)

In [ ]:
#first 100 names
networkResult = buildNetworkDict(result)

print("Number of unique nodes: %s"%len(networkResult['nodes']))
print("Number of links: %s"%len(networkResult['links']))

In [ ]:
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2021-02-28_CorruptionNetwork100Names4.json"
with open(jsonFile, 'w') as file:
    json.dump(networkResult, file) 